<a href="https://colab.research.google.com/github/fsheikh/QawwalRang/blob/colab_notebooks/colabs/update_qawwali_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dependencies setup

In [ ]:
import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")
print("Hello World from Colab Jupyter Notebook")

In [3]:
!pip install -Uqq ffmpy

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.0 MB/s eta 0:00:00


In [16]:
!pip install --upgrade --force-reinstall "git+https://github.com/ytdl-org/youtube-dl.git"

  Cloning https://github.com/ytdl-org/youtube-dl.git to /tmp/pip-req-build-9bxnp79w
  Running command git clone --filter=blob:none --quiet https://github.com/ytdl-org/youtube-dl.git /tmp/pip-req-build-9bxnp79w
  Resolved https://github.com/ytdl-org/youtube-dl.git to commit 4416f82c809a81737d68875dcb201e366d58dabd
  Preparing metadata (setup.py) ... done
  Created wheel for youtube-dl: filename=youtube_dl-2021.12.17-py2.py3-none-any.whl size=1949005 sha256=3b986fa829bbf67d4dc223e8ac7c0d192c8e55dfd5cf3fa91b6513aecf33157c
  Stored in directory: /tmp/pip-ephem-wheel-cache-va08h9k5/wheels/64/8e/40/866e846163e3e4859bbe820ff8847ec97f4320864f1525aa9a
Successfully built youtube-dl
  Attempting uninstall: youtube-dl
    Found existing installation: youtube-dl 2021.12.17
    Uninstalling youtube-dl-2021.12.17:
      Successfully uninstalled youtube-dl-2021.12.17


In [12]:
!wget https://raw.githubusercontent.com/fsheikh/QawwalRang/main/metadata/qawali_metadata.json

--2024-02-18 19:18:41--  https://raw.githubusercontent.com/fsheikh/QawwalRang/main/metadata/qawali_metadata.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22422 (22K) [text/plain]
Saving to: ‘qawali_metadata.json’

qawali_metadata.jso 100%[===================>]  21.90K  --.-KB/s    in 0.001s  

2024-02-18 19:18:41 (17.6 MB/s) - ‘qawali_metadata.json’ saved [22422/22422]



### Parse json and build local dataset

In [1]:
from ffmpy import FFmpeg as ffm
import json
import logging
import os
from pathlib import Path
import subprocess
from youtube_dl import YoutubeDL

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(message)s')
logger = logging.getLogger(__name__)

FormatSuffix = '.mp3'
def download_progress(prog):
    if prog['status'] == 'downloading':
      logger.info("Download prgogress: {}".format(prog['_percent_str']))
      return

def download(song, folder):
  logger.info("{}".format(song))
  if song['name'] == None or song['url'] == None:
    logger.error("Cannot download song {} from file {}".format(song['name'], song['url']))
    return
  else:
    logger.info("Download requested for {} from {}".format(song['name'], song['url']))
    out_song = folder / song['fid']
    tmp_song = folder / song['name']
    if out_song.with_suffix(FormatSuffix).exists():
      logger.info("song {} already exists, skipping...".format(str(tmp_song), str(out_song)))
      return
    # only download song with youtube links, ignore rest, they will also be converted
    # corresponding youtube links
    if not tmp_song.with_suffix(FormatSuffix).exists():
      if "youtube" in song['url']:
        ydl_params = {
          'format': 'bestaudio/best',
          'postprocessors': [{
          'key': 'FFmpegExtractAudio', 'preferredcodec' : 'mp3', 'preferredquality' : '128'
          }],
          'logger' : logger,
          'noplaylist' : True,
          'progress_hooks' : [download_progress]
          }
        ydl_params['outtmpl'] = str(tmp_song.with_suffix(FormatSuffix))
        ydl_opts = {'nocheckcertificate':True}
        with YoutubeDL(ydl_params) as ydl:
          try:
            ydl.download([song['url']])
          except Exception as e:
            logger.error("Failed to download from youtube with error: {}".format(e))
            return
        # We should have a downloaded audio file, trim it to required duration
        try:
          arg_start = str(song['start'])
          arg_duration = str(song['duration'])
          trimmer = ffm(inputs={str(tmp_song.with_suffix(FormatSuffix)): [ '-ss', arg_start]},
                        outputs={str(out_song.with_suffix(FormatSuffix)): ['-t', arg_duration]}
                       )
          print(trimmer.cmd)
          t_stdout, t_stderr = trimmer.run(input_data=None, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        except Exception as e:
          logger.error("Failed to trim downloaded song {} with error: {}".format(str(tmp_song), e))
      else:
        logger.warning("No other source than youtube supported!")
      # remove full downloaded file afterwards
      try:
        os.remove(str(tmp_song.with_suffix(FormatSuffix)))
      except OSError:
        logger.warning("Ignoring non existent file:{}".format(tmp_song))

dataDir = Path("./data")
dataDir.mkdir(parents=True, exist_ok=True)
with open('qawali_metadata.json') as j_file:
  qmap = json.load(j_file)
for qawwali in qmap['qawalian']:
  download(qawwali, dataDir)

ffmpeg -ss 0 -i data/amada_ba_qatal.mp3 -t 60 data/q001.mp3


ERROR:__main__:ERROR: Video unavailable
This video is no longer available because the YouTube account associated with this video has been terminated.
ERROR:__main__:Failed to download from youtube with error: ERROR: Video unavailable
This video is no longer available because the YouTube account associated with this video has been terminated.


ffmpeg -ss 0 -i data/ae_wadah_shikan.mp3 -t 60 data/q003.mp3


ERROR:__main__:ERROR: Video unavailable
ERROR:__main__:Failed to download from youtube with error: ERROR: Video unavailable


ffmpeg -ss 0 -i data/ajab_andaz.mp3 -t 60 data/q005.mp3
ffmpeg -ss 0 -i data/awayen_tay_tenu_dasan.mp3 -t 60 data/q006.mp3
ffmpeg -ss 0 -i data/az_dair_mugha.mp3 -t 60 data/q007.mp3
ffmpeg -ss 0 -i data/balaghal_sabri.mp3 -t 60 data/q008.mp3
ffmpeg -ss 0 -i data/dekh_lsm.mp3 -t 60 data/q009.mp3
ffmpeg -ss 0 -i data/ghonghat_chak_sajna.mp3 -t 60 data/q010.mp3
ffmpeg -ss 0 -i data/guftam_kay_roshan.mp3 -t 60 data/q011.mp3
ffmpeg -ss 0 -i data/hairat_e_ishq.mp3 -t 60 data/q012.mp3
ffmpeg -ss 0 -i data/haqeeqat_ka_agar.mp3 -t 60 data/q013.mp3
ffmpeg -ss 0 -i data/har_lehza_bashakal.mp3 -t 60 data/q014.mp3
ffmpeg -ss 0 -i data/inj_vichray_murr.mp3 -t 60 data/q015.mp3


...

ERROR:__main__:Failed to trim downloaded song data/yaad_e_mustafa with error: `ffmpeg -ss 0 -i data/yaad_e_mustafa.mp3 -t 60 data/q069.mp3` exited with status 1

STDOUT:


STDERR:
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enab

ffmpeg -ss 0 -i data/yaad_e_mustafa.mp3 -t 60 data/q069.mp3
ffmpeg -ss 0 -i data/mera_dil_meri_jaan.mp3 -t 60 data/q070.mp3


ERROR:__main__:Failed to trim downloaded song data/mera_dil_meri_jaan with error: `ffmpeg -ss 0 -i data/mera_dil_meri_jaan.mp3 -t 60 data/q070.mp3` exited with status 1

STDOUT:


STDERR:
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshin

ffmpeg -ss 0 -i data/mera_dhol_mahi.mp3 -t 60 data/q071.mp3
ffmpeg -ss 0 -i data/bekhud_kiye_daitay.mp3 -t 60 data/q072.mp3


ERROR:__main__:Failed to trim downloaded song data/bekhud_kiye_daitay with error: `ffmpeg -ss 0 -i data/bekhud_kiye_daitay.mp3 -t 60 data/q072.mp3` exited with status 1

STDOUT:


STDERR:
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshin